# Cattle Milk Yield Prediction - Complete Methodology

**Competition**: Kaggle Cattle Milk Yield Prediction  
**Objective**: Predict milk yield (liters) for 30,000 cattle based on 35 features  
**Evaluation Metric**: Root Mean Squared Error (RMSE)

## Dataset
- **Training**: 220,000 records
- **Test**: 30,000 records  
- **Features**: 35 (numerical and categorical)

## Journey Overview
This notebook documents the complete iterative process:
- **Initial CatBoost**: 4.16355 RMSE on Kaggle (11th place)
- **+ Feature Engineering**: 4.16148 RMSE (15th place)
- **+ Hyperparameter Tuning**: 4.16037 RMSE (15th place, BEST)
- **Expected Final**: ~4.151 RMSE (training on 100% data)

This notebook shows all experiments including what DIDN'T work (ensemble failures, Month categorical disaster).

## Section 1: Setup and Data Loading

We'll start by importing necessary libraries and loading the training and test datasets.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
print("Libraries imported successfully!")

In [ ]:
# Load data
train_df = pd.read_csv('cattle_data_train.csv')
test_df = pd.read_csv('cattle_data_test.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"\nFirst few rows:")
train_df.head()

## Section 2: Data Exploration

Before building models, we need to understand our data: distributions, missing values, correlations, and patterns.

### Dataset Overview
We have 220,000 training records with 35 features covering:
- **Cattle characteristics**: Age, Weight, Breed, Parity (number of births)
- **Farm management**: Management_System, Farm_ID, Housing_Score
- **Feeding**: Feed_Type, Feed_Quantity, Water_Intake, Grazing_Duration
- **Health**: Vaccination records, Mastitis status, Body_Condition_Score
- **Environment**: Climate_Zone, Temperature, Humidity
- **Lactation**: Lactation_Stage, Days_in_Milk, Previous_Week_Avg_Yield

In [ ]:
# Dataset overview
print("Dataset Information:")
print(train_df.info())
print("\n" + "="*80 + "\n")
print("Statistical Summary:")
train_df.describe()

In [ ]:
# Target variable distribution
plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.hist(train_df['Milk_Yield_L'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Milk Yield (Liters)', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title('Distribution of Milk Yield', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 3, 2)
train_df['Milk_Yield_L'].plot(kind='box')
plt.ylabel('Milk Yield (Liters)', fontsize=11)
plt.title('Box Plot - Milk Yield', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 3, 3)
plt.hist(train_df['Milk_Yield_L'], bins=50, cumulative=True, density=True, 
         edgecolor='black', alpha=0.7, color='green')
plt.xlabel('Milk Yield (Liters)', fontsize=11)
plt.ylabel('Cumulative Probability', fontsize=11)
plt.title('Cumulative Distribution', fontsize=12, fontweight='bold')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean Milk Yield: {train_df['Milk_Yield_L'].mean():.2f} L")
print(f"Median: {train_df['Milk_Yield_L'].median():.2f} L")
print(f"Std Dev: {train_df['Milk_Yield_L'].std():.2f} L")
print(f"Range: [{train_df['Milk_Yield_L'].min():.2f}, {train_df['Milk_Yield_L'].max():.2f}] L")
print(f"\nNote: Negative values present - will need to remove!")

In [ ]:
# Categorical features analysis
categorical_cols = ['Breed', 'Climate_Zone', 'Management_System', 'Lactation_Stage', 'Feed_Type']

fig, axes = plt.subplots(2, 3, figsize=(16, 9))
axes = axes.flatten()

for idx, col in enumerate(categorical_cols):
    train_df[col].value_counts().plot(kind='bar', ax=axes[idx], color='steelblue', edgecolor='black')
    axes[idx].set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('')
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(axis='y', alpha=0.3)

axes[-1].axis('off')
plt.tight_layout()
plt.show()

# Show value counts and percentages
for col in categorical_cols:
    print(f"\n{col}:")
    counts = train_df[col].value_counts()
    pcts = (counts / len(train_df) * 100).round(2)
    for val, count, pct in zip(counts.index, counts.values, pcts.values):
        print(f"  {val:20s}: {count:7,} ({pct:5.2f}%)")

### Key Observations from Exploration

**Target Variable (Milk_Yield_L)**:
- Mean: ~15.6 liters per cow per day
- Distribution is roughly normal but has **negative values** (impossible - must remove!)
- Range: -5.7 to 44.5 liters (negative values are data quality issues)

**Categorical Features**:
- **Breed**: 4 main breeds (Holstein dominant at ~50%)
- **Climate_Zone**: 6 zones (Mediterranean, Tropical, Arid, etc.)
- **Management_System**: 3 types (Intensive, Semi_Intensive, Mixed)
- **Lactation_Stage**: 3 stages (Early, Mid, Late - Mid is peak production)
- **Feed_Type**: 8 different feeds (Concentrates, Hay, Silage, etc.)

**Next Steps**: Deep data quality analysis to identify and fix issues.

## Section 3: Data Quality Analysis

Identifying data quality issues: missing values, outliers, invalid entries, typos, and feature correlations with target.

In [ ]:
# Missing values analysis
missing_counts = train_df.isnull().sum()
missing_pct = (missing_counts / len(train_df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing_counts[missing_counts > 0],
    'Percentage': missing_pct[missing_counts > 0]
}).sort_values('Missing_Count', ascending=False)

print("Missing Values Summary:")
print(missing_df)
print(f"\nTotal features with missing values: {len(missing_df)}")
print(f"\nMost significant:")
print(f"  - Feed_Quantity_kg: {missing_counts['Feed_Quantity_kg']:,} missing ({missing_pct['Feed_Quantity_kg']:.2f}%)")

In [ ]:
# Data quality issues
print("Critical Data Quality Issues:\n")
print("="*80)

# Issue 1: Negative milk yields
negative_yields = train_df[train_df['Milk_Yield_L'] < 0]
print(f"\n1. NEGATIVE MILK YIELDS (IMPOSSIBLE!)")
print(f"   Count: {len(negative_yields)} records ({len(negative_yields)/len(train_df)*100:.3f}%)")
print(f"   Range: {negative_yields['Milk_Yield_L'].min():.2f} to {negative_yields['Milk_Yield_L'].max():.2f} L")
print(f"   Decision: REMOVE these records (data entry errors)")

# Issue 2: Breed typos
print(f"\n2. BREED COLUMN TYPOS & WHITESPACE")
breed_counts = train_df['Breed'].value_counts()
print(f"   Unique values found: {train_df['Breed'].nunique()}")
for breed in breed_counts.index:
    print(f"   '{breed}': {breed_counts[breed]:,} records")
print(f"   Issues detected:")
print(f"     - 'Holstien' (typo) should be 'Holstein'")
print(f"     - ' Brown Swiss' (leading space)")
print(f"     - 'Brown Swiss ' (trailing space)")
print(f"   Decision: Strip whitespace and fix typo")

# Issue 3: Feed_Quantity missing
print(f"\n3. MISSING FEED_QUANTITY_KG")
print(f"   Count: {train_df['Feed_Quantity_kg'].isnull().sum():,} records (4.99%)")
print(f"   Decision: Impute with median by Feed_Type (domain knowledge)")
print(f"   Rationale: Different feed types have different typical quantities")

In [ ]:
# Feature correlation analysis
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('Cattle_ID')  # Remove ID column
if 'Milk_Yield_L' in numeric_cols:
    numeric_cols.remove('Milk_Yield_L')  # Remove target

# Calculate correlations with target
correlations = train_df[numeric_cols + ['Milk_Yield_L']].corr()['Milk_Yield_L'].drop('Milk_Yield_L')
correlations = correlations.abs().sort_values(ascending=False)

print("Feature Correlation Analysis with Milk Yield:\n")
print("="*80)
print("\nTop 15 Features (Highest Correlation):")
print(correlations.head(15))

print("\n" + "="*80)
print("Low Correlation Features (|corr| < 0.05) - Candidates for Removal:\n")
low_corr = correlations[correlations < 0.05]
print(low_corr)
print(f"\nTotal low-correlation features to remove: {len(low_corr)}")
print(f"\nRationale: Features with <0.05 correlation add noise without predictive value")

## Section 4: Data Cleaning

Based on our analysis, we'll systematically fix all data quality issues:
1. Fix Breed typos and whitespace
2. Remove negative milk yields (74 records)
3. Impute missing Feed_Quantity_kg values (10,481 records)
4. Remove 16 low-correlation features

Each decision is justified by either data quality requirements or statistical analysis.

In [ ]:
# Create working copies
train_clean = train_df.copy()
test_clean = test_df.copy()

print("CLEANING STEP 1: Fix Breed Typos & Whitespace")
print("="*80)
print(f"Before cleaning:")
print(train_clean['Breed'].value_counts())

# Strip whitespace and fix typo
train_clean['Breed'] = train_clean['Breed'].str.strip().replace('Holstien', 'Holstein')
test_clean['Breed'] = test_clean['Breed'].str.strip().replace('Holstien', 'Holstein')

print(f"\nAfter cleaning:")
print(train_clean['Breed'].value_counts())
print(f"\n✓ Fixed: 7 breed values → 4 correct breeds")

In [ ]:
print("\n" + "="*80)
print("CLEANING STEP 2: Remove Negative Milk Yields")
print("="*80)
print(f"Before: {len(train_clean):,} records")
print(f"Negative yields: {len(train_clean[train_clean['Milk_Yield_L'] < 0])}")

train_clean = train_clean[train_clean['Milk_Yield_L'] >= 0].copy()

print(f"After: {len(train_clean):,} records")
print(f"Removed: {len(train_df) - len(train_clean)} records ({(len(train_df) - len(train_clean))/len(train_df)*100:.3f}%)")
print(f"\n✓ All milk yields now ≥ 0")

In [ ]:
print("\n" + "="*80)
print("CLEANING STEP 3: Impute Missing Feed_Quantity_kg")
print("="*80)
print(f"Missing in train: {train_clean['Feed_Quantity_kg'].isnull().sum():,}")
print(f"Missing in test: {test_clean['Feed_Quantity_kg'].isnull().sum():,}")

# Calculate median by Feed_Type from TRAINING data only (prevent data leakage)
feed_medians = train_clean.groupby('Feed_Type')['Feed_Quantity_kg'].median()
print(f"\nMedian Feed Quantity by Feed_Type (from training data):")
for feed_type in feed_medians.sort_values(ascending=False).index:
    print(f"  {feed_type:20s}: {feed_medians[feed_type]:.2f} kg")

# Impute using these medians
for feed_type in feed_medians.index:
    train_clean.loc[(train_clean['Feed_Type'] == feed_type) & 
                    (train_clean['Feed_Quantity_kg'].isnull()), 'Feed_Quantity_kg'] = feed_medians[feed_type]
    test_clean.loc[(test_clean['Feed_Type'] == feed_type) & 
                   (test_clean['Feed_Quantity_kg'].isnull()), 'Feed_Quantity_kg'] = feed_medians[feed_type]

print(f"\nAfter imputation:")
print(f"  Missing in train: {train_clean['Feed_Quantity_kg'].isnull().sum()}")
print(f"  Missing in test: {test_clean['Feed_Quantity_kg'].isnull().sum()}")
print(f"\n✓ All Feed_Quantity values imputed")

In [ ]:
print("\n" + "="*80)
print("CLEANING STEP 4: Remove Low-Correlation Features")
print("="*80)

# Features to remove (16 total)
features_to_remove = [
    'FMD_Vaccine', 'Brucellosis_Vaccine', 'HS_Vaccine', 'BQ_Vaccine',
    'Anthrax_Vaccine', 'IBR_Vaccine', 'BVD_Vaccine', 'Rabies_Vaccine',
    'Mastitis', 'Humidity_percent', 'Housing_Score', 'Feeding_Frequency',
    'Resting_Hours', 'Rumination_Time_hrs', 'Walking_Distance_km',
    'Feed_Quantity_lb'  # Duplicate of Feed_Quantity_kg
]

print(f"Removing {len(features_to_remove)} features:")
for i, feat in enumerate(features_to_remove, 1):
    print(f"  {i:2d}. {feat}")

train_clean = train_clean.drop(columns=features_to_remove)
test_clean = test_clean.drop(columns=features_to_remove)

print(f"\nFinal shape:")
print(f"  Training: {train_clean.shape} (from {train_df.shape})")
print(f"  Test: {test_clean.shape} (from {test_df.shape})")
print(f"\n✓ Removed {len(features_to_remove)} features, kept {train_clean.shape[1]-1} predictive features")

### Data Cleaning Rationale

**1. Breed Typos & Whitespace**
- **Issue**: 'Holstien' vs 'Holstein', leading/trailing spaces
- **Impact**: Model would treat same breed as different categories
- **Solution**: String normalization (strip + replace)
- **Result**: 7 dirty values → 4 clean breeds

**2. Negative Milk Yields**
- **Issue**: 74 records with negative milk yields (-5.7 to -0.015 L)
- **Why impossible**: Cows cannot produce negative milk (physics!)
- **Likely cause**: Data entry error or sensor malfunction
- **Solution**: Remove these records
- **Impact**: Only 0.035% of data lost, worth it for data integrity

**3. Missing Feed_Quantity_kg**
- **Issue**: 10,481 missing values (4.99% of data)
- **Why not drop**: Losing 5% of training data hurts model performance
- **Solution**: Impute with median by Feed_Type
- **Rationale**: Different feeds have different typical quantities:
  - Concentrates: High-calorie, smaller volume (~12.04 kg)
  - Pasture Grass: Low-calorie, larger volume (~12.00 kg)
  - Median is robust to outliers
- **Critical**: Use training statistics only (no data leakage!)

**4. Low-Correlation Features**
- **Criterion**: |correlation| < 0.05 with target
- **16 features removed**: Vaccines (8), housing/feeding metrics (7), duplicate (1)
- **Why remove**: These add noise without predictive signal
- **Benefit**: Simpler model, faster training, less overfitting
- **Final**: 19 features (46% reduction from 35 original)

## Section 5: Feature Engineering

Creating new features based on domain knowledge about cattle milk production. This was a **critical breakthrough** that improved Kaggle score from 4.16355 → 4.16148!

### Features Created (9 total + Season):
1. **Weight_Age_Ratio**: Larger cattle relative to age = healthier = more milk
2. **Feed_Efficiency**: Feed intake per kg body weight (metabolism indicator)
3. **Water_Feed_Ratio**: Hydration critical for milk (milk is 87% water!)
4. **Peak_Lactation**: Binary flag for Mid-lactation stage (peak production)
5. **Heat_Stress**: Temperature indicator (heat reduces milk)
6. **Yield_Momentum**: Previous week's average (strong predictor)
7. **Age_Parity_Ratio**: Experience per birth (older cows more efficient)
8-9. **Month_Sin, Month_Cos**: Cyclical seasonal patterns
10. **Season**: 4-level categorical (Spring/Summer/Fall/Winter)

**Note**: We initially tried Month as categorical (12 levels) but it caused **catastrophic overfitting** (+0.11 RMSE on Kaggle!). Season with 4 levels works much better.

In [ ]:
def engineer_features(df):
    """
    Create 9 engineered features + Season based on domain knowledge
    """
    df_eng = df.copy()
    
    # 1. Weight-to-Age Ratio
    df_eng['Weight_Age_Ratio'] = df_eng['Weight_kg'] / (df_eng['Age_Months'] + 1)
    
    # 2. Feed Efficiency  
    df_eng['Feed_Efficiency'] = df_eng['Feed_Quantity_kg'] / (df_eng['Weight_kg'] + 1)
    
    # 3. Water-to-Feed Ratio
    df_eng['Water_Feed_Ratio'] = df_eng['Water_Intake_L'] / (df_eng['Feed_Quantity_kg'] + 1)
    
    # 4. Peak Lactation Indicator
    df_eng['Peak_Lactation'] = (df_eng['Lactation_Stage'] == 'Mid').astype(int)
    
    # 5. Heat Stress
    df_eng['Heat_Stress'] = df_eng['Ambient_Temperature_C']
    
    # 6. Yield Momentum
    df_eng['Yield_Momentum'] = df_eng['Previous_Week_Avg_Yield']
    
    # 7. Age-Parity Ratio
    df_eng['Age_Parity_Ratio'] = df_eng['Age_Months'] / (df_eng['Parity'] + 1)
    
    # 8-9. Month cyclical encoding
    df_eng['Date'] = pd.to_datetime(df_eng['Date'])
    df_eng['Month'] = df_eng['Date'].dt.month
    df_eng['Month_Sin'] = np.sin(2 * np.pi * df_eng['Month'] / 12)
    df_eng['Month_Cos'] = np.cos(2 * np.pi * df_eng['Month'] / 12)
    
    # 10. Season (4 levels - CRITICAL: NOT Month!)
    df_eng['Season'] = df_eng['Month'].map({
        12: 'Winter', 1: 'Winter', 2: 'Winter',
        3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Fall', 10: 'Fall', 11: 'Fall'
    })
    
    return df_eng

# Apply feature engineering
print("Applying feature engineering...")
train_eng = engineer_features(train_clean)
test_eng = engineer_features(test_clean)

print(f"\nEngineered Features Created:")
new_features = ['Weight_Age_Ratio', 'Feed_Efficiency', 'Water_Feed_Ratio',
                'Peak_Lactation', 'Heat_Stress', 'Yield_Momentum',
                'Age_Parity_Ratio', 'Month_Sin', 'Month_Cos', 'Season']
for i, feat in enumerate(new_features, 1):
    print(f"  {i:2d}. {feat}")

print(f"\nTotal features now: {len(train_eng.columns)}")